In [ ]:
from keras.utils import image_dataset_from_directory
import tensorflow as tf
train_dir_1 = 'trainning/train1'
train_dir_2 = 'trainning/train2'
train_dir_3 = 'trainning/train3'
validation_dir = 'train4' # Validation
train_dir_5 = 'trainning/train5'
test_dir = 'test'

trainning = [train_dir_1, train_dir_2,train_dir_3,train_dir_5]

train_dir = train_dir_2
IMG_SIZE = 32 # 32x32

# image_dataset_from_directory with labels="inferred" for 
# getting the images in the subdirectories and translating the subdirectory as a class 
# of type categorical
#train_dataset = image_dataset_from_directory(train_dir,image_size=(IMG_SIZE, IMG_SIZE),batch_size=32, labels="inferred", label_mode="categorical")
test_dataset = image_dataset_from_directory(test_dir,image_size=(IMG_SIZE, IMG_SIZE), labels="inferred",label_mode="categorical")
validation_dataset = image_dataset_from_directory(validation_dir,image_size=(IMG_SIZE, IMG_SIZE), labels="inferred",label_mode="categorical")

train_dataset = tf.data.Dataset

for i in trainning:
    if i == trainning[0]:
        train_dataset = image_dataset_from_directory(i, image_size=(IMG_SIZE, IMG_SIZE), labels="inferred", label_mode="categorical")
        continue
    train_dataset = train_dataset.concatenate( image_dataset_from_directory(i, image_size=(IMG_SIZE, IMG_SIZE), labels="inferred", label_mode="categorical"))




In [ ]:
#imports
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import keras
from keras import layers
import numpy as np
IMG_SIZE = 32

inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

x = layers.Rescaling(1./255)(inputs)

x = layers.Conv2D(filters=32, kernel_size=3, activation="relu", padding='same', kernel_regularizer=keras.regularizers.l2(0.001))(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Dropout(0.25)(x)

x = layers.Conv2D(filters=64, kernel_size=3, activation="relu", padding='same', kernel_regularizer=keras.regularizers.l2(0.001))(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Dropout(0.25)(x)

x = layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding='same', kernel_regularizer=keras.regularizers.l2(0.001))(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Dropout(0.25)(x)


x = layers.Flatten()(x)
x = layers.Dense(256, activation="relu", kernel_regularizer=keras.regularizers.l2(0.001))(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation="relu", kernel_regularizer=keras.regularizers.l2(0.001))(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)

# Output layer
outputs = layers.Dense(10, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Callbacks
checkpoint = ModelCheckpoint(
    'models_S/S_without_DA_First.h5', monitor='val_loss', verbose=0,
    save_best_only=True, mode='min'
)

early_stopping = EarlyStopping(
    monitor='val_loss', patience=5, verbose=1, mode='min',
    restore_best_weights=True
)

rl = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=4,
    verbose=0,
    mode="min",
)


# batch size 32
# Test loss: 0.6775595545768738
# Test accuracy: 0.8391000032424927

# batch size 64
# Test loss: 0.693570613861084
# Test accuracy: 0.8421000242233276

# batchsize 128
# Test loss: 0.7256516814231873
# Test accuracy: 0.8317000269889832

# sem BatchNormalization
#Test loss: 0.7569888234138489
# Test accuracy: 0.8009999990463257

# sem Droupout
#Test loss: 1.2996251583099365
#Test accuracy: 0.7465000152587891



#64 BATCH_SIZE - pacience 10

# Epoch 102: early stopping
# 313/313 [==============================] - 1s 3ms/step - loss: 0.7004 - accuracy: 0.8350
# Test loss: 0.7004380226135254
# Test accuracy: 0.8349999785423279

#128
# Test loss: 0.7256516814231873
# Test accuracy: 0.8317000269889832

In [ ]:
history = model.fit(
    train_dataset,
    epochs=200,
    batch_size=128,
    validation_data=validation_dataset,
    callbacks=[checkpoint, early_stopping, rl]
  )


# Avaliar o modelo (com o teste)
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau
import keras
from keras import layers

data_augmentation = keras.Sequential(
        [layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),])

    # rotation_range=20,
    # width_shift_range=0.1,
    # height_shift_range=0.1,
    # zoom_range=0.2,
    # horizontal_flip=True,
    # fill_mode='nearest',
    # # shear_range=0.2,
    # # zoom_range=0.2,
    # # horizontal_flip=True,
    # # fill_mode='nearest',
    # # channel_shift_range=0.2,

inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)

x = layers.Conv2D(filters=32, kernel_size=3, activation="relu", padding='same', kernel_regularizer=keras.regularizers.l2(0.001))(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Dropout(0.25)(x)

x = layers.Conv2D(filters=64, kernel_size=3, activation="relu", padding='same', kernel_regularizer=keras.regularizers.l2(0.001))(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Dropout(0.25)(x)

x = layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding='same', kernel_regularizer=keras.regularizers.l2(0.001))(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Dropout(0.25)(x)

x = layers.Flatten()(x)
x = layers.Dense(256, activation="relu", kernel_regularizer=keras.regularizers.l2(0.001))(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation="relu", kernel_regularizer=keras.regularizers.l2(0.001))(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)

# Output layer
outputs = layers.Dense(10, activation="softmax")(x)

model2 = keras.Model(inputs=inputs, outputs=outputs)

# compilar o modelo
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Callbacks
checkpoint = ModelCheckpoint(
    'models_S/S_with_DA_First.h5', monitor='val_loss', verbose=0,
    save_best_only=True, mode='min'
)

early_stopping = EarlyStopping(
    monitor='val_loss', patience=4, verbose=1, mode='min',
    restore_best_weights=True
)

learning_rate = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=4,
    verbose=0,
    mode="min",
)

In [ ]:
history2 = model.fit(
    train_dataset,
    epochs=200,
    batch_size=128,
    validation_data=validation_dataset,
    callbacks=[checkpoint, early_stopping, rl]
  )


# Avaliar o modelo (com o teste)
test_loss, test_accuracy = model2.evaluate(test_dataset)
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')

In [ ]:
from keras.models import load_model
model = load_model('/content/best_modelS_WITHOUT_64.h5')


In [ ]:
import pandas as pd
import numpy as np

# usar para instalar...: pip install scikit-learn
from sklearn import metrics
from sklearn.model_selection import train_test_split

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense

from sklearn.preprocessing import LabelEncoder

# to install IPython, use: pip install ipython
from IPython.display import display

from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report


# Definir nomes das classes
class_names = {
    0: "Airplane",
    1: "Automobile",
    2: "Bird",
    3: "Cat",
    4: "Deer",
    5: "Dog",
    6: "Frog",
    7: "Horse",
    8: "Ship",
    9: "Truck"
}

# Prever para todo o conjunto de teste
y_pred = []
y_true = []

for x, y in test_dataset:
    predictions = model.predict(x)
    y_pred.extend(np.argmax(predictions, axis=1))
    y_true.extend(np.argmax(y, axis=1))

y_pred = np.array(y_pred)
y_true = np.array(y_true)

# Matriz de confusão
conf_matrix = confusion_matrix(y_true, y_pred)

# Plotar a matriz de confusão
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names.values(), yticklabels=class_names.values())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Relatório de classificação
class_report = classification_report(y_true, y_pred, target_names=class_names.values())
print("Classification Report:\n", class_report)


In [ ]:
def graph(history, title):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, acc, 'b', label='Training accuracy')
    plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

graph(history, "Modelo S sem Data Augmentation")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming you have already trained your model and obtained predictions and true labels
# Example lists for demonstration


# Extract images and labels from test_dataset
test_images = []
test_labels = []

for images, labels in test_dataset:
    test_images.append(images.numpy())  # Assuming you convert images to numpy arrays
    test_labels.append(labels.numpy())

test_images = np.concatenate(test_images)
test_labels = np.concatenate(test_labels)

# Normalize the images to [0, 1]
test_images = test_images.astype(np.float32) / 255.0

# Display images with predictions and true labels
plt.figure(figsize=(12, 12))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(test_images[i], cmap=plt.cm.binary)  # Display the ith image
    predicted_label = class_names[y_pred[i]]
    true_label = class_names[y_true[i]]
    if predicted_label == true_label:
        color = 'green'
    else:
        color = 'red'
    plt.xlabel(f'Pred: {predicted_label} \n True: {true_label}', color=color)

plt.show()


In [ ]:
import matplotlib.pyplot as plt
from keras.models import Model
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
import random

# Assuming you have defined your test_dataset and model somewhere

# Get the first image from test_dataset
first_batch = next(iter(test_dataset))  # Get the first batch from the dataset
first_image = first_batch[0][0]  # Extract the first image from the batch

# Display the original image
plt.imshow(first_image.numpy().astype("uint8"))
plt.title('Original Image')
plt.axis('off')
plt.show()

# Preprocess the image for visualization
img = first_image.numpy()
img = np.expand_dims(img, axis=0)  # Now img has shape (1, 32, 32, 3)

def visualize_filters(model, img):
    layer_outputs = [layer.output for layer in model.layers if 'conv' in layer.name or 'pool' in layer.name]

    activation_model = Model(inputs=model.input, outputs=layer_outputs)

    activations = activation_model.predict(img)

    layer_names = [layer.name for layer in model.layers if 'conv' in layer.name or 'pool' in layer.name]

    for layer_name, layer_activation in zip(layer_names, activations):
        n_features = layer_activation.shape[-1]
        size = layer_activation.shape[1]

        n_cols = n_features // 16  # Number of columns in the grid
        display_grid = np.zeros((size * n_cols, size * 16))

        for col in range(n_cols):
            for row in range(16):
                channel_image = layer_activation[0, :, :, col * 16 + row]
                channel_image -= channel_image.mean()
                channel_image /= (channel_image.std() + 1e-5)
                channel_image *= 64
                channel_image += 128
                channel_image = np.clip(channel_image, 0, 255).astype('uint8')
                display_grid[col * size : (col + 1) * size, row * size : (row + 1) * size] = channel_image

        scale = 1. / size
        plt.figure(figsize=(scale * display_grid.shape[1], scale * display_grid.shape[0]))
        plt.title(layer_name)
        plt.grid(False)
        plt.imshow(display_grid, aspect='auto', cmap='viridis')

# Visualize filters for the model and the first image
visualize_filters(model, img)


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import cv2

# Assuming test_dataset and model are already defined

random_index = random.randint(0, 500)

first_batch = next(iter(test_dataset))  # Get the first batch from the dataset
img = first_batch[0][0]  # Extract the first image from the batch

img_array = img.numpy()
img_array = np.expand_dims(img_array, axis=0)  # Now img_array has shape (1, 32, 32, 3)

plt.imshow(img_array[0].astype("uint8"))  # Display the original image
plt.title('Original Image')
plt.axis('off')
plt.show()

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    grads = tape.gradient(class_channel, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

last_conv_layer_name = 'conv2d_1'  # Replace with your actual last conv layer name
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

plt.matshow(heatmap)
plt.title('Grad-CAM Heatmap')
plt.show()

def superimpose_heatmap(img, heatmap, alpha=0.4):
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = heatmap * alpha + img
    return np.uint8(superimposed_img)

# Convert img_array to (32, 32, 3) for superimpose_heatmap function
img_for_superimpose = img_array[0].astype("uint8")

superimposed_img = superimpose_heatmap(img_for_superimpose, heatmap)

plt.imshow(superimposed_img)
plt.title('Image with Grad-CAM Heatmap')
plt.axis('off')
plt.show()


In [ ]:
# Function to extract images and labels from a tf.data.Dataset
from sklearn.decomposition import PCA

def get_images_and_labels(dataset, num_samples):
    dataset = dataset.unbatch().take(num_samples)
    images = []
    labels = []
    for img, label in dataset:
        images.append(img.numpy())
        labels.append(label.numpy())
    return np.array(images), np.array(labels)

# Extract images and labels from the test dataset
x_test, y_test = get_images_and_labels(test_dataset, num_samples=1000)

# Visualization function
def visualize_feature_space(layer_name, x_data, y_data):
    feature_extractor = tf.keras.models.Model(inputs=model.input, outputs=model.get_layer(layer_name).output)

    num_samples = len(x_data)
    sample_indices = np.random.choice(len(x_data), num_samples, replace=False)
    sample_images = x_data[sample_indices]

    features = feature_extractor.predict(sample_images)

    pca = PCA(n_components=2)
    features_reduced = pca.fit_transform(features.reshape(num_samples, -1))

    plt.figure(figsize=(8, 6))
    plt.scatter(features_reduced[:, 0], features_reduced[:, 1], c=np.argmax(y_data[sample_indices], axis=1), cmap='tab10', marker='o', alpha=0.8)
    plt.colorbar(label='Class')
    plt.title(f'Feature Space (PCA) - Layer {layer_name}')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.grid(True)
    plt.show()

# Assuming you have x_test and y_test as your test dataset
layer_name = 'conv2d_1'
visualize_feature_space(layer_name, x_test, y_test)

In [ ]:
model.summary()